In [30]:
import pandas as pd
from sklearn import tree
import pydotplus
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report


from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import numpy as np
from sklearn import metrics

In [31]:
# Import the two datasets 

df = pd.read_csv("log_labeled_trans.csv", header = 0).fillna(0)
df

,Start time,End time,Hall-Bedroom door,Hall-Bathroom door,ToiletFlush,Plates cupboard,Fridge,Microwave,Groceries Cupboard,Hall-Toilet door,Frontdoor,Cups cupboard,Pans Cupboard,Freezer,Dishwasher,Washingmachine,Label,Label_ID,Activity_Size
0,2008-02-25 09:33:00,2008-02-25 09:33:00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,go to bed,10,1.0
1,2008-02-25 09:37:00,2008-02-25 09:38:00,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,use toilet,4,4.0
2,2008-02-25 09:49:00,2008-02-25 09:53:00,0.0,0.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,prepare Breakfast,13,8.0
3,2008-02-25 10:02:00,2008-02-25 10:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,take shower,5,2.0
4,2008-02-25 10:19:00,2008-02-25 16:54:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,leave house,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,2008-03-21 09:42:00,2008-03-21 15:50:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,leave house,1,2.0
241,2008-03-21 15:52:00,2008-03-21 15:53:00,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,use toilet,4,4.0
242,2008-03-21 17:03:00,2008-03-21 18:05:00,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,go to bed,10,4.0
243,2008-03-21 18:24:00,2008-03-21 18:25:00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,use toilet,4,4.0


In [32]:
data_top = df.columns
columns = list(data_top) 

In [33]:
features = columns
features.remove("Label")
features.remove("Label_ID")
features.remove("Start time")
features.remove("End time")
features

['Hall-Bedroom door',
 'Hall-Bathroom door',
 'ToiletFlush',
 'Plates cupboard',
 'Fridge',
 'Microwave',
 'Groceries Cupboard',
 'Hall-Toilet door',
 'Frontdoor',
 'Cups cupboard',
 'Pans Cupboard',
 'Freezer',
 'Dishwasher',
 'Washingmachine',
 'Activity_Size']

In [34]:
X = df[features]
y = df.Label_ID

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size=0.3, random_state=1) # 70% training and 30% test
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)
data = tree.export_graphviz(clf, out_file=None, feature_names=features)
graph = pydotplus.graph_from_dot_data(data)
graph.write_png('mydecisiontree_classifier_test.png')

#Predict the response for test dataset
y_pred = clf.predict(X_test)
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# img=pltimg.imread('mydecisiontree.png')
# imgplot = plt.imshow(img)
# plt.show() 


Accuracy: 0.8783783783783784


In [36]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        10
           4       0.94      0.89      0.91        36
           5       1.00      0.91      0.95        11
          10       0.56      0.83      0.67         6
          13       0.62      1.00      0.77         5
          15       0.00      0.00      0.00         2
          17       1.00      0.75      0.86         4

    accuracy                           0.88        74
   macro avg       0.73      0.77      0.74        74
weighted avg       0.88      0.88      0.87        74



C:\Users\gdfe\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gdfe\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gdfe\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
print("Trying to predict the take shower activity, with ID 5, \nwith the following combination 1,0,0,0,0,0,0,3,0,0,0,0,0,0")
pred = clf.predict([[1,0,0,0,0,0,0,3,0,0,0,0,0,0]])
print("Value predicted:", pred)
if(pred == 5): print("Correct prediction") 
else: print("Wrong predicition")

Trying to predict the take shower activity, with ID 5, 
with the following combination 1,0,0,0,0,0,0,3,0,0,0,0,0,0
Value predicted: [10]
Wrong predicition
